Script using the specsanalyzer class/library to remove the grid from the data via Fourrier Filtering.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from specsanalyzer.img_tools import fourier_filter_2d
import xarray as xr
import ast

ModuleNotFoundError: No module named 'specsanalyzer'

'pip' is not recognized as an internal or external command,
operable program or batch file.


Defining the run. Only 2D data. Can work for 3D as well, have to implement some iterating over the other axis

In [ ]:
import calendar

year = 2022
month = 12
day = 6
run = 2977

path = f"Y:\TRARPES\Converted_h5_files\{year}\{month:0>2} {calendar.month_name[month]}\Day {day:0>2}\{str(run)}.h5"
save_path = f"Y:\TRARPES\Converted_h5_files\{year}\{month:0>2} {calendar.month_name[month]}\Day {day:0>2}\{str(run)}_gridrem.h5"

data_set = xr.open_dataset(path, engine="h5netcdf")

In [ ]:
data_set

Reading out the FFT peaks used for the filtering from the metadata

In [ ]:
peaks = ast.literal_eval(data_set.attrs['fft_filter_peaks'])
peaks

Performing the FFT and plotting FFT, plot the mask and plot the filtered FFT

In [ ]:
fft = fourier_filter_2d(data_set.spectrum, peaks, ret="fft")
mask = fourier_filter_2d(data_set.spectrum, peaks, ret="mask")
filtered_fft = fourier_filter_2d(data_set.spectrum, peaks, ret="filtered_fft")

fig, ax = plt.subplots(1, 3, figsize=(15, 5))
im0 = ax[0].imshow(np.abs(fft[0:350, 0:180]), vmax=300000000)
ax[0].set_title("FFT")
fig.colorbar(im0, ax=ax[0])
im1 = ax[1].imshow(np.abs(mask[0:350, 0:180]), vmax=1)
ax[1].set_title("Mask")
fig.colorbar(im1, ax=ax[1])
im2 = ax[2].imshow(np.abs(filtered_fft[0:350, 0:180]), vmax=300000000)
ax[2].set_title("Filtered FFT")
fig.colorbar(im2, ax=ax[2])
for a in ax:
    a.set_xlabel("Energy / Pixels")
    a.set_ylabel("Angle / Pixels")
    

Perform the FFT filtering and plot the inital data, the filtered data and the difference image

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(21, 5))

filtered = fourier_filter_2d(data_set.spectrum, peaks)
im0 = ax[0].imshow(data_set.spectrum)
ax[0].set_title("Before FFT")
fig.colorbar(im0, ax=ax[0])
im1 = ax[1].imshow(filtered)
ax[1].set_title("After FFT")
fig.colorbar(im1, ax=ax[1])
im2 = ax[2].imshow(data_set.spectrum - filtered)
ax[2].set_title("Difference")
fig.colorbar(im2, ax=ax[2])

Overwrite the spectrum of the inital DataSet with the filtered spectrum. Plot difference for confirmation -> should be 0 everywhere.

In [ ]:
filtered_set = data_set.copy()
filtered_set.spectrum.values = filtered
plt.imshow(filtered_set.spectrum - filtered)
plt.colorbar()
plt.title("Difference after overwriting spectrum")

In [ ]:
filtered_set

Save the filtered dataset in a new h5 file. The path is defined in the first cell.

In [ ]:
save = False
if save:
    filtered_set.to_netcdf(path=save_path, mode='w', format='NETCDF4', engine='h5netcdf')